# PMR3508 - Aprendizado de Máquina e Reconhecimento de Padrões
## Base Costa Rican Household

### Import das bibliotecas

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt

### Leitura dos dados

In [ ]:
dfTrain = pd.read_csv("../input/train.csv",
          sep=r'\s*,\s*',
          engine='python',
          na_values="")

In [ ]:
dfTest = pd.read_csv("../input/test.csv",
         sep=r'\s*,\s*',
         engine='python',
         na_values="")

### Tratamento dos dados

In [ ]:
# dfTrain = dfTrain.dropna() # Retirada de NA
dfTrain = dfTrain.apply(lambda x: x.fillna(x.value_counts().index[0])) # Substituicao pelo mais frequente

In [ ]:
# dfTest = dfTest.dropna() # Retirada de NA
dfTest = dfTest.apply(lambda x: x.fillna(x.value_counts().index[0])) # Substituicao pelo mais frequente

### Análise dos dados

In [ ]:
dfTrain.shape

In [ ]:
dfTrain.describe()

In [ ]:
dfTrain.select_dtypes(exclude='number').head()

Como as features string não podem ser analisadas facilmente, precisam ser pré-processadas

In [ ]:
strList = ["Id", "idhogar", "dependency", "edjefe", "edjefa"]
dfAll = pd.concat([dfTrain[strList], dfTest[strList]]).apply(preprocessing.LabelEncoder().fit_transform)
bkpDfTrain = dfTrain.copy()
bkpDfTest = dfTest.copy()
dfTrain[strList] = dfAll.iloc[:dfTrain.shape[0]]
dfTest[strList] = dfAll.iloc[dfTrain.shape[0]:]
dfTrain.describe()

In [ ]:
labels = dfTrain.columns
labels = labels.drop("Target")
dfTrain[labels] = (dfTrain[labels] - dfTrain[labels].mean()) / dfTrain[labels].std()
dfTest[labels] = (dfTest[labels] - dfTrain[labels].mean()) / dfTrain[labels].std()
dfTrain.corr()["Target"].sort_values()

hogar_nin, r4t1, SQBhogar_nin, meaneduc, cielorazo e escolari são as features mais interessantes a serem utilizadas

### Classificação

In [ ]:
features = ["hogar_nin", "r4t1", "SQBhogar_nin", "meaneduc", "cielorazo", "escolari"]

In [ ]:
XTrain = dfTrain[features]
YTrain = bkpDfTrain.Target
XTest = dfTest[features]

In [ ]:
scores = []
for i in range(1, 200, 10):
    knn = KNeighborsClassifier(n_neighbors=i)
    scores.append(cross_val_score(knn, XTrain, YTrain, cv=10).mean())
plt.plot(range(1, 200, 10), scores)
knn = KNeighborsClassifier(n_neighbors=81) # Melhor resultado

In [ ]:
knn.fit(XTrain, YTrain)

In [ ]:
YTest = knn.predict(XTest)
YTest

In [ ]:
dfSave = pd.DataFrame(data={"Id" : bkpDfTest["Id"], "Target" : YTest})
pd.DataFrame(dfSave[["Id", "Target"]], columns = ["Id", "Target"]).to_csv("Output.csv", index=False)